# **Check for GPU Presence and Useability**

In [2]:
!nvidia-smi

Thu Aug  7 16:57:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.43                 Driver Version: 566.43         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   49C    P8              4W /   72W |     283MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# **Importing required packages**

In [4]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# **Setting The Configurations (Hyperparameters for this Model)**

In [6]:
CONFIG = {
    'csv_file_path': r"C:\Users\Webbies\Jupyter_Notebooks\LLM_FineTune\Website_Data_Samples_1000.csv",
    'train_ratio': 0.8,
    'val_ratio': 0.1, 
    'd_model': 256,
    'nhead': 4,
    'num_encoder_layers': 2,
    'num_decoder_layers': 2,
    'dim_feedforward': 512,
    'dropout': 0.1,
    'num_epochs': 450, # This need to be increased for better performance with early stopping parameter
    'batch_size': 8,  # Increase batch size for more data
    'learning_rate': 1e-4,
    'pad_token': 2,
    'sos_token': 0,
    'eos_token': 1,
    'max_seq_len': 5500,
    'patience': 10, # New: Number of epochs to wait for improvement
    'min_delta': 0.001, # New: Minimum change in validation loss to be considered an improvement
    'model_save_path': 'AI_WebsiteDesigner_LLM.pth'
}

<h1 style = "color:green">Future Modifications That are Needed for this Complete Process</h1>

* Set the Number of Epochs to 300 or 400 with an early stopping parameter to stop the model from overfitting. Setting the epoch number to higher is needed because currently with 200 epochs, it is observed that for 200 epochs the training loss is still decreasing. 
* Increase the number of batch size from 4 to 16 or even 32 to make the model efficient by speeding the train process
* Increase the value of the 'max_seq_len' parameter from 512 to 1024 or more to generate long codes
* **All this modifications will lead to greater training process and will consume more memory and GPU**
* Currently, this model building process utilizes F1 score and BLUE score that matches tokens between generated output and actual output. It performs more efficiently than the normal accuracy score that calculates percentage match as missing a single white space or comma can yield drastic mismatch. 

# **Download necessary NLTK data (this only needs to be run once)**

In [9]:
try:
    nltk.data.find('tokenizers/punkt')
    print("Data Already Found")
except nltk.downloader.DownloadError:
    print("Downloading 'punkt' data for NLTK...")
    nltk.download('punkt')

Data Already Found


# **Use GPU if available**

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# **Data Loading and Preprocessing**

In [13]:
def load_process_split_data(csv_file_path, train_ratio, val_ratio):
    """
    Loads data from a CSV and performs a train-validation-test split.
    """
    try:
        df = pd.read_csv(csv_file_path)
    except FileNotFoundError:
        print(f"Error: The file '{csv_file_path}' was not found. Please ensure it is in the correct directory.")
        return None, None, None, None, None

    df.rename(columns={'llm_generated_idea': 'prompt', 'text': 'code'}, inplace=True)

    # Split into train+val and test sets
    train_val_df, test_df = train_test_split(df, test_size=1-train_ratio, random_state=42)

    # Split train+val into train and validation sets
    val_size = int(len(df) * val_ratio)
    train_df, val_df = train_test_split(train_val_df, test_size=val_size, random_state=42)

    train_data = list(zip(train_df['prompt'], train_df['code']))
    val_data = list(zip(val_df['prompt'], val_df['code']))
    test_data = list(zip(test_df['prompt'], test_df['code']))

    prompts = df['prompt'].tolist()
    codes = df['code'].tolist()

    return train_data, val_data, test_data, prompts, codes

# **Building the Complete Vocabulary for this Project**

In [78]:
def build_vocab(prompts, codes):
    """
    Builds a character-level vocabulary from the prompts and codes.
    """
    prompt_vocab = set(" ".join(prompts))
    code_vocab = set(" ".join(codes))

    prompt_char_to_idx = {char: i + 3 for i, char in enumerate(sorted(list(prompt_vocab)))}
    code_char_to_idx = {char: i + 3 for i, char in enumerate(sorted(list(code_vocab)))}

    special_tokens = {
        '<sos>': CONFIG['sos_token'],
        '<eos>': CONFIG['eos_token'],
        '<pad>': CONFIG['pad_token']
    }
    prompt_char_to_idx.update(special_tokens)
    code_char_to_idx.update(special_tokens)

    prompt_idx_to_char = {idx: char for char, idx in prompt_char_to_idx.items()}
    code_idx_to_char = {idx: char for char, idx in code_char_to_idx.items()}

    max_prompt_len = max(len(p) for p in prompts) + 2  # +2 for <sos> and <eos>
    max_code_len = max(len(c) for c in codes) + 2     # +2 for <sos> and <eos>

    print(f"Prompt vocab size: {len(prompt_char_to_idx)}")
    print(f"Code vocab size: {len(code_char_to_idx)}")
    print(f"Maximum prompt length: {max_prompt_len}")
    print(f"Maximum code length: {max_code_len}")

    return (prompt_char_to_idx, prompt_idx_to_char,
            code_char_to_idx, code_idx_to_char,
            max_prompt_len, max_code_len)

# **The Tokenization Process**

In [80]:
def tokenize(text, char_to_idx, add_sos=True, add_eos=True):
    """Converts a text string into a list of token indices."""
    tokens = [char_to_idx[char] for char in text]
    if add_sos:
        tokens.insert(0, CONFIG['sos_token'])
    if add_eos:
        tokens.append(CONFIG['eos_token'])
    return tokens

# **Pytorch Dataset and DataLoader & The Collate Function**

In [82]:
class NLPtoCodeDataset(Dataset):
    def __init__(self, data, prompt_char_to_idx, code_char_to_idx):
        self.data = data
        self.prompt_char_to_idx = prompt_char_to_idx
        self.code_char_to_idx = code_char_to_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        prompt, code = self.data[idx]
        prompt_tensor = torch.tensor(tokenize(prompt, self.prompt_char_to_idx), dtype=torch.long)
        code_tensor = torch.tensor(tokenize(code, self.code_char_to_idx), dtype=torch.long)
        return prompt_tensor, code_tensor

In [84]:
def collate_fn(batch):
    """
    Pads sequences to the same length within a batch.
    """
    prompts, codes = zip(*batch)
    prompts_padded = pad_sequence(prompts, batch_first=True, padding_value=CONFIG['pad_token'])
    codes_padded = pad_sequence(codes, batch_first=True, padding_value=CONFIG['pad_token'])
    return prompts_padded.to(device), codes_padded.to(device)

# **Transformer Model Definition**

In [86]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len):  # max_len is no longer a hardcoded default
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [110]:
class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, nhead,
                 num_encoder_layers, num_decoder_layers, dim_feedforward,
                 dropout, max_src_len, max_tgt_len):  # Added max_src_len and max_tgt_len
        super().__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)

        # Instantiate PositionalEncoding with the dynamic max lengths
        self.pos_encoder_src = PositionalEncoding(d_model, dropout, max_len=max_src_len)
        self.pos_encoder_tgt = PositionalEncoding(d_model, dropout, max_len=max_tgt_len)

        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None, src_padding_mask=None, tgt_padding_mask=None):
        src = self.pos_encoder_src(self.src_embedding(src))
        tgt = self.pos_encoder_tgt(self.tgt_embedding(tgt))
        output = self.transformer(src, tgt, src_mask=src_mask, tgt_mask=tgt_mask,
                                  src_key_padding_mask=src_padding_mask,
                                  tgt_key_padding_mask=tgt_padding_mask)
        return self.fc_out(output)

# **Training Code**

In [90]:
def create_masks(src, tgt):
    src_padding_mask = (src == CONFIG['pad_token'])
    tgt_padding_mask = (tgt == CONFIG['pad_token'])
    tgt_len = tgt.size(1)
    tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt_len).to(device)
    return src_padding_mask, tgt_padding_mask, tgt_mask

In [92]:
def train_one_epoch(model, dataloader, optimizer, criterion):
    """Performs a single training loop over the dataset."""
    model.train()
    total_loss = 0
    for src_batch, tgt_batch in dataloader:
        tgt_input = tgt_batch[:, :-1]
        tgt_output = tgt_batch[:, 1:]

        src_padding_mask, tgt_padding_mask, tgt_mask = create_masks(src_batch, tgt_input)

        output = model(src_batch, tgt_input, src_mask=None, tgt_mask=tgt_mask,
                       src_padding_mask=src_padding_mask, tgt_padding_mask=tgt_padding_mask)

        loss = criterion(output.view(-1, output.size(-1)), tgt_output.reshape(-1))
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

    return total_loss / len(dataloader)

# **Defineing the Evaluation Loss Function to Calculate Evaluate Loss**

In [94]:
def evaluate_loss(model, dataloader, criterion):
    """Calculates the loss on a validation or test set."""
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for src_batch, tgt_batch in dataloader:
            tgt_input = tgt_batch[:, :-1]
            tgt_output = tgt_batch[:, 1:]
            
            src_padding_mask, tgt_padding_mask, tgt_mask = create_masks(src_batch, tgt_input)

            output = model(src_batch, tgt_input, src_mask=None, tgt_mask=tgt_mask,
                           src_padding_mask=src_padding_mask, tgt_padding_mask=tgt_padding_mask)
            
            loss = criterion(output.view(-1, output.size(-1)), tgt_output.reshape(-1))
            total_loss += loss.item()
            
    return total_loss / len(dataloader)

In [96]:
def translate(model, prompt, prompt_char_to_idx, code_idx_to_char):
    """Generates code from a given prompt using the trained model."""
    model.eval()
    src_tokens = tokenize(prompt, prompt_char_to_idx)
    src_tensor = torch.tensor(src_tokens, dtype=torch.long).unsqueeze(0).to(device)
    tgt_tokens = [CONFIG['sos_token']]

    with torch.no_grad():
        for _ in range(CONFIG['max_seq_len']):
            tgt_tensor = torch.tensor(tgt_tokens, dtype=torch.long).unsqueeze(0).to(device)
            _, _, tgt_mask = create_masks(src_tensor, tgt_tensor)
            output = model(src_tensor, tgt_tensor, tgt_mask=tgt_mask)
            next_token_logits = output[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1).item()
            if next_token == CONFIG['eos_token']:
                break
            tgt_tokens.append(next_token)

    generated_code = "".join([code_idx_to_char[token] for token in tgt_tokens[1:] if token != CONFIG['eos_token']])
    return generated_code

In [98]:
def evaluate_metrics(model, dataloader, prompt_char_to_idx, code_idx_to_char):
    """
    Evaluates the model's performance on the test set using various metrics.
    """
    model.eval()
    all_predicted_tokens = []
    all_target_tokens = []
    
    bleu_scores = []
    
    with torch.no_grad():
        for src_batch, tgt_batch in dataloader:
            for i in range(src_batch.size(0)):
                # Get ground truth
                ground_truth_tokens = tgt_batch[i].cpu().numpy()
                ground_truth_tokens = ground_truth_tokens[ground_truth_tokens != CONFIG['pad_token']]
                
                # Get prompt
                prompt_tokens = src_batch[i].cpu().numpy()
                prompt_tokens = prompt_tokens[prompt_tokens != CONFIG['pad_token']]
                
                # Decode prompt to string for the `translate` function
                prompt_str = "".join([prompt_idx_to_char[t] for t in prompt_tokens if t not in [CONFIG['sos_token'], CONFIG['eos_token'], CONFIG['pad_token']]])
                
                # Generate prediction
                generated_code_str = translate(model, prompt_str, prompt_char_to_idx, code_idx_to_char)
                
                # Tokenize generated code for comparison
                predicted_tokens = tokenize(generated_code_str, code_char_to_idx, add_sos=True, add_eos=True)
                
                # Align the sequences for token-level metrics
                all_predicted_tokens.extend(predicted_tokens)
                all_target_tokens.extend(ground_truth_tokens)
                
                # Calculate BLEU score
                reference = [[code_idx_to_char[t] for t in ground_truth_tokens if t not in [CONFIG['sos_token'], CONFIG['eos_token'], CONFIG['pad_token']]]]
                candidate = list(generated_code_str)
                if candidate:
                    bleu_scores.append(sentence_bleu(reference, candidate, smoothing_function=SmoothingFunction().method1))

    # Calculate token-level F1, Precision, and Recall
    min_len = min(len(all_predicted_tokens), len(all_target_tokens))
    f1 = f1_score(all_target_tokens[:min_len], all_predicted_tokens[:min_len], average='micro')
    precision = precision_score(all_target_tokens[:min_len], all_predicted_tokens[:min_len], average='micro')
    recall = recall_score(all_target_tokens[:min_len], all_predicted_tokens[:min_len], average='micro')

    # Calculate average BLEU score
    avg_bleu = np.mean(bleu_scores) if bleu_scores else 0

    return avg_bleu, f1, precision, recall

# **The Early Stopping Class**

In [100]:
class EarlyStopping:
    """
    Early stops the training if validation loss doesn't improve after a given patience.
    """
    def __init__(self, patience=5, min_delta=0, path='checkpoint.pth'):
        self.patience = patience
        self.min_delta = min_delta
        self.path = path
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.save_checkpoint(model)
        else:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True

    def save_checkpoint(self, model):
        """Saves model when validation loss decreases."""
        torch.save(model.state_dict(), self.path)

# **The Main Execution Block Starts from Here**

In [102]:
# 1. Load and split data
train_data, val_data, test_data, prompts, codes = load_process_split_data(CONFIG['csv_file_path'], CONFIG['train_ratio'], CONFIG['val_ratio'])
if train_data is None:
    print("Exiting due to data loading failure.")
    exit()

In [104]:
# 2. Build vocabulary
(prompt_char_to_idx, prompt_idx_to_char,code_char_to_idx, code_idx_to_char,max_prompt_len, max_code_len) = build_vocab(prompts, codes)

src_vocab_size = len(prompt_char_to_idx)
tgt_vocab_size = len(code_char_to_idx)

Prompt vocab size: 71
Code vocab size: 98
Maximum prompt length: 476
Maximum code length: 4805


In [106]:
# 3. Initialize datasets and dataloaders
train_dataset = NLPtoCodeDataset(train_data, prompt_char_to_idx, code_char_to_idx)
val_dataset = NLPtoCodeDataset(val_data, prompt_char_to_idx, code_char_to_idx)
test_dataset = NLPtoCodeDataset(test_data, prompt_char_to_idx, code_char_to_idx)

train_dataloader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False, collate_fn=collate_fn)

In [112]:
# 4. Initialize model, optimizer, and loss function
model = TransformerModel(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=CONFIG['d_model'],
    nhead=CONFIG['nhead'],
    num_encoder_layers=CONFIG['num_encoder_layers'],
    num_decoder_layers=CONFIG['num_decoder_layers'],
    dim_feedforward=CONFIG['dim_feedforward'],
    dropout=CONFIG['dropout'],
    max_src_len=max_prompt_len,
    max_tgt_len=max_code_len
).to(device)

In [114]:
# 5.1 Setting the Optimizer and The Criterion
optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG['learning_rate'])
criterion = nn.CrossEntropyLoss(ignore_index=CONFIG['pad_token'])

# 5.2 Initialize early stopping
early_stopping = EarlyStopping(patience=CONFIG['patience'],min_delta=CONFIG['min_delta'],path=CONFIG['model_save_path'])

In [39]:
# 6. Training loop
print("Starting training...")
for epoch in range(CONFIG['num_epochs']):
    train_loss = train_one_epoch(model, train_dataloader, optimizer, criterion)
    val_loss = evaluate_loss(model, val_dataloader, criterion)

    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{CONFIG['num_epochs']}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

    # Check for early stopping
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print(f"Early stopping triggered at epoch {epoch+1}!")
        break

print("Training complete.")

Starting training...
Epoch 10/450: Train Loss = 1.8878, Val Loss = 1.8298
Epoch 20/450: Train Loss = 1.5760, Val Loss = 1.4956
Epoch 30/450: Train Loss = 1.3638, Val Loss = 1.2855
Epoch 40/450: Train Loss = 1.2238, Val Loss = 1.1471
Epoch 50/450: Train Loss = 1.1220, Val Loss = 1.0432
Epoch 60/450: Train Loss = 1.0416, Val Loss = 0.9660
Epoch 70/450: Train Loss = 0.9739, Val Loss = 0.9033
EarlyStopping counter: 1 of 10
Epoch 80/450: Train Loss = 0.9157, Val Loss = 0.8486
EarlyStopping counter: 1 of 10
Epoch 90/450: Train Loss = 0.8676, Val Loss = 0.7981
EarlyStopping counter: 1 of 10
EarlyStopping counter: 1 of 10
Epoch 100/450: Train Loss = 0.8228, Val Loss = 0.7518
Epoch 110/450: Train Loss = 0.7825, Val Loss = 0.7136
EarlyStopping counter: 1 of 10
Epoch 120/450: Train Loss = 0.7497, Val Loss = 0.6817
EarlyStopping counter: 1 of 10
EarlyStopping counter: 1 of 10
EarlyStopping counter: 1 of 10
Epoch 130/450: Train Loss = 0.7177, Val Loss = 0.6514
EarlyStopping counter: 1 of 10
EarlySt

In [40]:
# 7. Save the Model Checkpoint path in desired folder
print("\n--- Saving the trained model ---")
torch.save(model.state_dict(), CONFIG['model_save_path'])
print(f"Model saved to {CONFIG['model_save_path']}")


--- Saving the trained model ---
Model saved to AI_WebsiteDesigner_LLM.pth


In [52]:
# 9. Generate and display a sample output
print("\n--- Final Inference Example ---")
test_prompt, ground_truth_code = test_data[0]
generated_code = translate(model, test_prompt, prompt_char_to_idx, code_idx_to_char)

print(f"Test Prompt: '{test_prompt}'")
print("-" * 50)
print(f"Generated Code: \n{generated_code}")
print("-" * 50)
print(f"Ground Truth Code:\n{ground_truth_code}")


--- Final Inference Example ---
Test Prompt: 'Tech Company: A minimalist design with a large, central hero image, navigation menu on the top, and a left sidebar for information about the team, services, and contact details. The footer can have a simple layout with social media icons and copyright information.'
--------------------------------------------------
Generated Code: 
<html>
<link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
<body class="bg-gray-100">
  <header class="bg-white p-4 flex justify-center">
    <img src="https://source.unsplash.com/random/100x50/?logo" alt="Logo" class="h-10">
   </header>

  <nav class="bg-white p-4 flex justify-center">
    <ul class="flex space-x-4">
      <li><a href="#" class="text-gray-800 hover:text-gray-800">Home</a></li>
       <li><a href="#" class="text-gray-600 hover:text-gray-800">About</a></li>
        <li><a href="#" class="text-gray-800 hover:text-gray-800">About</a></li>
        <l

In [116]:
# 8. Getting the Model Evaluation Scores (F1 score and BLUE)
avg_bleu, f1, precision, recall = evaluate_metrics(model, test_dataloader, prompt_char_to_idx, code_idx_to_char)
        
print("\n--- Final Evaluation Results ---")
print(f"Average BLEU Score: {avg_bleu:.4f}")
print(f"Token-level F1 Score: {f1:.4f}")
print(f"Token-level Precision: {precision:.4f}")
print(f"Token-level Recall: {recall:.4f}")

KeyboardInterrupt: 

In [70]:
import webbrowser
import os

# **Getting The preview of the Actual HTML Code**

In [73]:
html_code = ground_truth_code

# Create a temporary HTML file
file_name = "Actual.html"
with open(file_name, "w") as f:
    f.write(html_code)

# Open the file in the default web browser
webbrowser.open(file_name)

True

# **Getting The Preview of the Generated HTML Code**

In [76]:
html_code = generated_code

# Create a temporary HTML file
file_name = "Generated.html"
with open(file_name, "w") as f:
    f.write(html_code)

# Open the file in the default web browser
webbrowser.open(file_name)

True